# Identify Missing Time Blocks

In [2]:
import pandas as pd
import sqlite3

Import full Ceusus Block Group data, generate one row per 3-hour block group per day.

In [4]:
df_census = pd.read_csv("../data/census-data/BlockGroup/FinalBlockGroupData.csv")

In [5]:
start_year = '2013'
end_year = '2017'

df_time = pd.DataFrame({
    'date': pd.date_range(
        start = pd.Timestamp(start_year),
        end = pd.Timestamp(end_year) + pd.offsets.YearEnd(0) + pd.Timedelta(days=1),
        freq = '3h',
        closed = 'left'
    )
})

Add columns for month/day/year/3-hr time block category column.

In [6]:
df_time['date'] = pd.to_datetime(df_time['date'])
df_time['year'] = df_time['date'].dt.year
df_time['month'] = df_time['date'].dt.month
df_time['day'] = df_time['date'].dt.day
df_time['time_block'] = pd.cut(df_time['date'].dt.hour,8,labels=['12am-3am','3am-6am','6am-9am','9am-12pm','12pm-3pm','3pm-6pm','6pm-9pm','9pm-12am'],include_lowest=True)

In [7]:
df_time.head(10)

,date,year,month,day,time_block
0,2013-01-01 00:00:00,2013,1,1,12am-3am
1,2013-01-01 03:00:00,2013,1,1,3am-6am
2,2013-01-01 06:00:00,2013,1,1,6am-9am
3,2013-01-01 09:00:00,2013,1,1,9am-12pm
4,2013-01-01 12:00:00,2013,1,1,12pm-3pm
5,2013-01-01 15:00:00,2013,1,1,3pm-6pm
6,2013-01-01 18:00:00,2013,1,1,6pm-9pm
7,2013-01-01 21:00:00,2013,1,1,9pm-12am
8,2013-01-02 00:00:00,2013,1,2,12am-3am
9,2013-01-02 03:00:00,2013,1,2,3am-6am


Merge with census on Year

In [8]:
full_date_time_census = df_time.merge(df_census[['Tract','BlockGroup','Year']], left_on='year', right_on='Year')

In [9]:
full_date_time_census.head(10)

,date,year,month,day,time_block,Tract,BlockGroup,Year
0,2013-01-01,2013,1,1,12am-3am,2202,3,2013
1,2013-01-01,2013,1,1,12am-3am,2301,1,2013
2,2013-01-01,2013,1,1,12am-3am,2301,2,2013
3,2013-01-01,2013,1,1,12am-3am,2302,1,2013
4,2013-01-01,2013,1,1,12am-3am,2400,1,2013
5,2013-01-01,2013,1,1,12am-3am,2400,2,2013
6,2013-01-01,2013,1,1,12am-3am,2400,3,2013
7,2013-01-01,2013,1,1,12am-3am,2400,4,2013
8,2013-01-01,2013,1,1,12am-3am,2501,1,2013
9,2013-01-01,2013,1,1,12am-3am,2501,2,2013


In [10]:
full_date_time_census.count()

date          6722520
year          6722520
month         6722520
day           6722520
time_block    6722520
Tract         6722520
BlockGroup    6722520
Year          6722520
dtype: int64

Import Census/Crime/Weather to figure out how to merge

In [15]:
conn = sqlite3.connect('../data/crime_census_weather_tod.db')
c = conn.cursor()
df = pd.read_sql('''select * from all_crimes where year >=2013 and year <= 2017''', conn)

In [16]:
df.head()

,level_0,BLOCK_GROUP,year,month,day,tod,crime_counts,apparent_temp,cloud_cover,dew_point,...,MedianAgeMargin,HousingUnits,HousingUnitsMargin,UnweightedSampleHousingUnits,State,County,Tract,BlockGroup,census_year,crime_rate
0,33,000100 1,2013,1,10,Early Morning,1,44.43,0.34,27.77,...,7.2,743.0,144.0,40.0,11.0,1.0,000100,1,2013,80.645161
1,34,000100 1,2013,1,30,Night,1,57.28,0.98,55.38,...,7.2,743.0,144.0,40.0,11.0,1.0,000100,1,2013,80.645161
2,35,000100 1,2013,2,4,Evening,1,32.68,0.91,18.64,...,7.2,743.0,144.0,40.0,11.0,1.0,000100,1,2013,80.645161
3,36,000100 1,2013,2,14,Afternoon,1,43.85,0.00,28.66,...,7.2,743.0,144.0,40.0,11.0,1.0,000100,1,2013,80.645161
4,37,000100 1,2013,3,8,Mid Afternoon,1,44.84,0.72,27.55,...,7.2,743.0,144.0,40.0,11.0,1.0,000100,1,2013,80.645161


In [17]:
list(df)

['level_0',
 'BLOCK_GROUP',
 'year',
 'month',
 'day',
 'tod',
 'crime_counts',
 'apparent_temp',
 'cloud_cover',
 'dew_point',
 'humidity',
 'percip_intensity',
 'percip_probability',
 'pressure',
 'temperature',
 'uv_index',
 'visibility',
 'wind_bearing',
 'wind_gust',
 'wind_speed',
 'index',
 'TotalPop',
 'TPopMargin',
 'UnWgtSampleCtPop',
 'PerCapitaIncome',
 'PerCapIncMargin',
 'MedianHouseholdInc',
 'MedHouseholdIncMargin',
 'MedianAge',
 'MedianAgeMargin',
 'HousingUnits',
 'HousingUnitsMargin',
 'UnweightedSampleHousingUnits',
 'State',
 'County',
 'Tract',
 'BlockGroup',
 'census_year',
 'crime_rate']

Add date/time/3-hr time fields to crime/census/weather df

In [18]:
df['START_DATE'] = pd.to_datetime(df['START_DATE'])

df['START_YEAR'] = df['START_DATE'].dt.year
df['START_MONTH'] = df['START_DATE'].dt.month
df['START_DAY'] = df['START_DATE'].dt.day

df['START_TIME_CATEGORY'] = pd.cut(df['START_DATE'].dt.hour,8,labels=['12am-3am','3am-6am','6am-9am','9am-12pm','12pm-3pm','3pm-6pm','6pm-9pm','9pm-12am'],include_lowest=True)

KeyError: 'START_DATE'

In [ ]:
df.head()

Need to update Block Group field in census/date/time df so it will join correctly to crime/weather/census

In [ ]:
full_date_time_census['Tract'] = full_date_time_census['Tract'].apply(str)

full_date_time_census['Tract'] = full_date_time_census['Tract'].apply(lambda x: x.zfill(6))

In [ ]:
full_date_time_census['BlockGroup_clean'] = full_date_time_census['Tract'] + " " + full_date_time_census['BlockGroup']

In [ ]:
full_date_time_census.head()

Joining crime/weather to full date/timeblock data

In [ ]:
time_blocks_and_crime = full_date_time_census.merge(df, how='left', left_on=['year','month','day','time_block','BlockGroup_clean'], right_on=['START_YEAR','START_MONTH','START_DAY','START_TIME_CATEGORY','BLOCK_GROUP'])

In [ ]:
time_blocks_and_crime.head()

In [ ]:
no_crime_time_census = time_blocks_and_crime[time_blocks_and_crime['OCTO_RECORD_ID'].isna()]

In [ ]:
no_crime_time_census.count()